# Supply Points (Case dell'Acqua) Data Preprocessing

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import seaborn as sns

# Paths

In [ ]:
utils_folder = os.path.join("..", "..", "utils")

with open(os.path.join(utils_folder, "onedrive.txt"), "r") as f:
    cloud_data_folder = os.path.join(f.readline().strip(), "Case dell'acqua")

grab_samples_folder = os.path.join(cloud_data_folder, "Grab Samples")
sensors_folder = os.path.join(cloud_data_folder, "Sensori")

local_data_folder = os.path.join("..", "..", "data")
clean_data_folder = os.path.join(local_data_folder, "Clean Data")
raw_data_folder = os.path.join(local_data_folder, "Raw Data")

plot_folder = os.path.join(local_data_folder, "Plots")

all_grab_samples_path = os.path.join(
    raw_data_folder, "Tutti punti - Grab Samples"
)

grab_samples_supply_points_path = os.path.join(
    raw_data_folder,
    "Case dell'acqua - Grab Samples (main)/0. Case acqua - 2010-2023.xlsx",
)

In [ ]:
# Tra i grab non c'è l'ORP, mentre
# tra i sensori non c'è DOC (c'è il TOC) e L'UVA254

# Quindi in comune abbiamo:
# Color, TOC, Nitrati, Turbidity, pH, Temperature, Conductivity, Free Chlorine

In [ ]:
columns_mapping = {
    'Cloro attivo libero (al prelievo)': 'Free Chlorine (mg/L)',
    'Colore': 'Color (CU)',
    'Concentrazione ioni idrogeno (al prelievo)': 'pH',
    'Conducibilità\xa0 elettrica 20°C': 'Conductivity (uS/cm)',
    'Carbonio organico totale': 'TOC (mg/L)',
    'Temperatura (al prelievo)': 'Temperature (°C)',
    'Nitrati': 'Nitrate (mg/L)',
    'Dibromoclorometano': 'Dibromochloromethane (ug/L)',
    'Bromoformio': 'Bromoform (ug/L)',
    'Cloroformio': 'Chloroform (ug/L)',
    'di cloro bromometano': 'Dichlorobromomethane (ug/L)',
}

# Load Grab Samples

In [ ]:
grab_df = pd.DataFrame()
for filename in os.listdir(grab_samples_folder):
    if filename.endswith(".csv"):
        if grab_df.empty:
            grab_df = pd.read_csv(os.path.join(grab_samples_folder, filename), sep=";")
        else:
            df = pd.read_csv(os.path.join(grab_samples_folder, filename), sep=';')
            grab_df = pd.concat([grab_df, df])

In [ ]:
grab_df.head(5)

In [ ]:
columns = list(columns_mapping.values())

In [ ]:
grab_df.drop(
    columns=[
        'CAMPIONE',
        'ANALITA',
        'VALORE',
        'UNITA\' DI MISURA',
    ],
    inplace=True
)


In [ ]:
grab_df = grab_df[grab_df['COMPONENTE'].isin(columns_mapping.keys())]

In [ ]:
grab_df['COMPONENTE'] = grab_df['COMPONENTE'].map(columns_mapping)

In [ ]:
grab_df = grab_df.pivot(
    index=['DATA CAMPIONAMENTO', 'INDIRIZZO'],
    columns='COMPONENTE',
    values='VALORE TESTO'
).reset_index()

In [ ]:
grab_df['DATA CAMPIONAMENTO'] = pd.to_datetime(grab_df['DATA CAMPIONAMENTO'], format='%d/%m/%Y %H:%M')

In [ ]:
grab_df.sort_values(
    by=['DATA CAMPIONAMENTO'],
    inplace=True
)

## Fix LOD values

In [ ]:
import re

def convert_string_values(s):
    
    try:
        s = s.strip()
        s = float(s)
    except Exception as e:
        pass
    
    if isinstance(s, (int, float)):
        return s
    elif pd.isna(s):
        return None
    else:
        if "," in s:
            s = s.replace(",", ".")
        if "<" in s:
            number = re.findall(r"\d+\.?\d*", s)
            return float(number[0]) / 2 if number else None
        elif ">" in s:
            number = re.findall(r"\d+\.?\d*", s)
            return float(number[0]) if number else None
        elif "*" in s or re.search("[a-zA-Z]", s):
            number = re.findall(r"\d+\.?\d*", s)
            return float(number[0]) if number else None
        else:
            return None

In [ ]:
def set_label(value):
    
    try:
        value = value.strip()
        value = float(value)
    except Exception as e:
        pass
    
    if pd.isna(value):
        return "NaN"
    elif isinstance(value, (int, float)):
        return "Normal"
    elif "<" in value:
        return "Less than"
    elif ">" in value:
        return "Greater than"
    else:
        return "NaN"

In [ ]:
for column in columns:
    label_column = column + "_label"
    grab_df.loc[:, label_column] = grab_df[column].apply(set_label)

In [ ]:
grab_df[columns] = grab_df[columns].map(
    convert_string_values
)

In [ ]:
grab_df

# Load Sensor Samples

In [ ]:
columns_mapping = {
    "Measurement interval=900[sec] (Export-Aggregation disabled)": "DateTime",
    "Measurement interval=999[sec] (Export-Aggregation disabled)": "DateTime",
    "Measurement interval=0[sec] (Export-Aggregation disabled)": "DateTime",
    "COLORtrue - Measured value [Hazen-eq.] (Limit:0.00-300.00)": "Color (CU)",
    "TOCeq - Measured value [mg/l] (Limit:0.00-22.00)": "TOC (mg/L)",
    "NO3eq - Measured value [mg/l] (Limit:0.00-88.00)": "Nitrate (mg/L)",
    # "UV254t - Measured value [Abs/m] (Limit:0.00-71.00)": "UVA254 (1/m)",
    "pH - Measured value (Limit:0.00-14.00)": "pH",
    "Temperature - Measured value [C] (Limit:-5.00-100.00)": "Temperature (°C)",
    "Conductivity - Measured value [uS/cm] (Limit:0.10-600000.00)": "Conductivity (uS/cm)",
    "Free Chlorine - Measured value [mg/l] (Limit:0.00-2.00)": "Free Chlorine (mg/L)",
}


In [ ]:
sensor_dict = {}

for sensor_file in os.listdir(sensors_folder):
    if sensor_file == ".DS_Store":
        continue

    sensor_folder = os.path.join(sensors_folder, sensor_file)
    for filename in os.listdir(sensor_folder):
        if not filename.endswith(".csv"):
            continue

        house_code = filename.split("_")[0]
        
        if filename.endswith(".csv"):
            if house_code not in sensor_dict:
                
                df = pd.read_csv(
                    os.path.join(sensor_folder, filename), header=1, sep=";"
                )
                df.rename(columns=columns_mapping, inplace=True)
                
                sensor_dict[house_code] = df
            else:
                df = pd.read_csv(os.path.join(sensor_folder, filename), header=1, sep=";")
                
                df.rename(columns=columns_mapping, inplace=True)
                
                sensor_dict[house_code] = pd.concat([sensor_dict[house_code], df])

In [ ]:
sensor_columns = set(columns_mapping.values())

for house_code, df in sensor_dict.items():
    sensor_dict[house_code] = sensor_dict[house_code][list(sensor_columns)]

# Processing

## Grab Samples

In [ ]:
grab_df.rename(
    columns={
        "DATA CAMPIONAMENTO": "DateTime",
        "INDIRIZZO": "Code",
    },
    inplace=True
)

In [ ]:
# change name of Punta di prelievo values to match codes
def change_name(name):
    if "Tognazzi" in name:
        return "Tognazzi"
    elif "Tabacchi" in name:
        return "Tabacchi"
    elif "Gramsci" in name:
        return "Gramsci"
    elif "Berna" in name:
        return "Berna"
    elif "Bande Nere" in name or "Piazzale Giovanni" in name:
        return "Bande Nere"
    elif "Prealpi" in name:
        return "Prealpi"
    elif "Chiostergi" in name:
        return "Chiostergi"
    elif "Montevideo" in name or "Montevid" in name:
        return "Montevideo"
    elif "Fortunato" in name:
        return "Fortunato"
    else:
        return name

In [ ]:
grab_df["Code"] = grab_df["Code"].map(change_name)

In [ ]:
grab_df

## Sensor Samples

In [ ]:
sensor_dict.keys()

In [ ]:
# change the name of the keys to match the names in the grab_df
sensor_dict["Tabacchi"] = sensor_dict.pop("via TABACCHI")
sensor_dict["Tognazzi"] = sensor_dict.pop("via Tognazzi")
sensor_dict["Prealpi"] = sensor_dict.pop("Piazza Prealpi")

In [ ]:
for code in grab_df["Code"].unique():
    sensor_df = sensor_dict.pop(code)
    sensor_df["DateTime"] = pd.to_datetime(sensor_df["DateTime"])
    sensor_df.set_index("DateTime", inplace=True)
    sensor_dict[code] = sensor_df

# Missing Values

## Grab

In [ ]:
# compute number of missing values for each column
for code in grab_df["Code"].unique():
    code_df = grab_df[grab_df["Code"] == code]
    for column in columns:
        # count the number of missing values
        missing_values = code_df[column].isna().sum()
        if missing_values > 0:
            print(
                f"{code} has {missing_values} missing values in column {column}"
            )

In [ ]:
grab_df

In [ ]:
# compute number of rows that have at least one missing value
for code in grab_df["Code"].unique():
    code_df = grab_df[grab_df["Code"] == code]

    missing_values = (
        code_df[columns]
        .isna()
        .any(axis=1)
        .sum()
    )
    if missing_values > 0:
        print(f"{code} has {missing_values} rows with missing values")

In [ ]:
# for the moment no imputation is done

## Sensor

In [ ]:
for code in sensor_dict.keys():
    sensor_df = sensor_dict[code]
    for column in sensor_df.columns:
        missing_values = sensor_df[column].isna().sum()
        if missing_values > 0:
            print(
                f"{code} has {missing_values} missing values in column {column}"
            )

In [ ]:
# the number of missing values is very low, so we can do implicit imputation with time interpolation
for code in sensor_dict.keys():
    sensor_df = sensor_dict.pop(code)
    sensor_df.interpolate(method="time", inplace=True)
    sensor_dict[code] = sensor_df

In [ ]:
grab_df.sort_values(by="DateTime", inplace=True)

# Add Further Sensor Data

In [ ]:
first_batch_sensor_df = pd.read_excel(
    os.path.join(clean_data_folder, "Riunione 24-04-2024", "Sensor Data.xlsx")
)

In [ ]:
first_batch_sensor_df

In [ ]:
first_batch_sensor_df.Code.unique()

In [ ]:
sensor_dict.keys()

In [ ]:
code_mapping = {
    'HOUSE_BANDENERE': 'Bande Nere',
    'HOUSE_BERNA': 'Berna',
    'HOUSE_CHIOSTERGI': 'Chiostergi',
    'HOUSE_FORTUNATO': 'Fortunato',
    'HOUSE_GRAMSCI': 'Gramsci',
    'HOUSE_MONTEVIDEO': 'Montevideo',
    'HOUSE_PREALPI': 'Prealpi',
    'HOUSE_TOGNAZZI': 'Tognazzi',
    'HOUSE_TABACCHI': 'Tabacchi',
}

In [ ]:
first_batch_sensor_df["Code"] = first_batch_sensor_df["Code"].map(code_mapping)

In [ ]:
first_batch_sensor_df.columns

In [ ]:
for code in first_batch_sensor_df["Code"].unique():
    df = first_batch_sensor_df[first_batch_sensor_df["Code"] == code].copy()
    df["DateTime"] = pd.to_datetime(df["DateTime"])
    df.set_index("DateTime", inplace=True)

    df.drop(columns=["Code"], inplace=True)

    sensor_df = sensor_dict.pop(code)

    sensor_df = pd.concat([sensor_df, df])
    sensor_df.sort_index(inplace=True)
    sensor_dict[code] = sensor_df

In [ ]:
sensor_dict["Berna"]

## Add Tabacchi Late Data

In [ ]:
tabacchi_folder = os.path.join(raw_data_folder, "Tabacchi Late")

In [ ]:
tabacchi_df = pd.DataFrame()

for filename in os.listdir(tabacchi_folder):
    if filename.endswith(".csv"):
        if tabacchi_df.empty:
            tabacchi_df = pd.read_csv(os.path.join(tabacchi_folder, filename), sep=";")
        else:
            df = pd.read_csv(os.path.join(tabacchi_folder, filename), sep=";")
            tabacchi_df = pd.concat([tabacchi_df, df])

In [ ]:
tabacchi_df

In [ ]:
tabacchi_df['Tag Name'] = tabacchi_df['Tag Name'].apply(lambda x: x.split("_")[-1])

In [ ]:
tabacchi_df['Tag Name'].unique()

In [ ]:
tabacchi_df.drop(
    columns=[
        'Chart',
        'Historian Tag Name',
        'Quality',
    ],
    inplace=True
)

In [ ]:
# Color, TOC, Nitrati, Turbidity, pH, Temperature, Conductivity, Free Chlorine are the ones we need

In [ ]:
columns_mapping = {
    'NO3eq': 'Nitrate (mg/L)',
    'Colore': 'Color (CU)',
    'TOCeq': 'TOC (mg/L)',
    'pH': 'pH',
    'Turbidity': 'Turbidity (NTU)',
    'Temperature': 'Temperature (°C)',
    'Conductivity': 'Conductivity (uS/cm)',
    'Chlorine': 'Free Chlorine (mg/L)',
    'SAC254': 'UVA254 (1/m)',
}

In [ ]:
tabacchi_df = tabacchi_df.pivot(
    index=['TimeStamp'],
    columns='Tag Name',
    values='Value'
).reset_index()

In [ ]:
columns_to_drop = tabacchi_df.columns.difference(list(columns_mapping.keys()) + ['TimeStamp']) 
tabacchi_df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
tabacchi_df.rename(columns=columns_mapping, inplace=True)
tabacchi_df.rename(
    columns={
        'TimeStamp': 'DateTime',
    },
    inplace=True
)

In [ ]:
tabacchi_df['DateTime'] = pd.to_datetime(tabacchi_df['DateTime'], format='mixed')
# remove the time zone information
tabacchi_df['DateTime'] = tabacchi_df['DateTime'].dt.tz_localize(None)

In [ ]:
tabacchi_df.sort_values(
    by=['DateTime'],
    inplace=True
)

In [ ]:
tabacchi_df.set_index("DateTime", inplace=True)

In [ ]:
# add the df to sensor_dict
df = sensor_dict.pop("Tabacchi")
df = pd.concat([df, tabacchi_df])

In [ ]:
df.sort_index(inplace=True)
sensor_dict["Tabacchi"] = df

# Outliers

## Sensor Samples

In [ ]:
# Notes

# - GRAMSCI
# Turbidity selected upper threshold is 1.5
# Conductivity selected lower threshold is 400
# Free Chlorine selected upper threshold is 0.8
# UVA254 selected upper threshold is 1.5


# - BERNA
# Turbidity selected upper threshold is 1.5
# Temperature selected upper threshold is 19.5
# Conductivity selected lower threshold is 400
# Free Chlorine selected upper threshold is 0.2

# - BANDE NERE
# Turbidity selected upper threshold is 1
# Conductivity selected lower threshold is 400
# Nitrate selected lower threshold is 20
# UVA254 selected upper threshold is 0.4

# - CHIOSTREGI
# free chlorine selected upper threshold is 0.06

# - FORTUNATO
# Turbidity selected upper threshold is 1
# Conductivity selected lower threshold is 400
# Nitrate selected lower threshold is 25
# UVA254 selected upper threshold is 0.4

# - MONTEVIDEO
# Color selected upper threshold is 4
# Turbidity selected upper threshold is 1
# Conductivity selected lower threshold is 400
# Free Chlorine selected upper threshold is 1
# Nitrate selected lower threshold is 20
# TOC selected upper threshold is 1
# UVA254 selected upper threshold is 4

# - PREALPI
# Turbidity selected upper threshold is 0.7
# UVA254 selected upper threshold is 1.5

# - TABACCHI

# - TOGNAZZI
# Conductivity selected lower threshold is 400
# Free Chlorine selected upper threshold is 0.4

thresholds = {
    "Gramsci": {
        "Turbidity (NTU)": 1.5,
        "Conductivity (uS/cm)": 400,
        # "Free Chlorine (mg/l)": 0.8,
        "UVA254 (1/m)": 1.5,
    },
    "Berna": {
        "Turbidity (NTU)": 1.5,
        "Temperature (°C)": 19.5,
        "Conductivity (uS/cm)": 400,
        # "Free Chlorine (mg/L)": 0.2,
        "UVA254 (1/m)": 1.5,
    },
    "Bande Nere": {
        "Turbidity (NTU)": 1,
        "Conductivity (uS/cm)": 400,
        "Nitrate (mg/L)": 20,
        "UVA254 (1/m)": 0.4,
    },
    "Chiostergi": {
        # "Free Chlorine (mg/L)": 0.06,
    },
    "Fortunato": {
        "Turbidity (NTU)": 1,
        "Conductivity (uS/cm)": 400,
        "Free Chlorine (mg/L)": 1.0,
        "Nitrate (mg/L)": 25,
        "UVA254 (1/m)": 0.4,
        "TOC (mg/L)": 1,
    },
    "Montevideo": {
        "Color (CU)": 4,
        "Turbidity (NTU)": 1,
        "Conductivity (uS/cm)": 400,
        "Free Chlorine (mg/L)": 1,
        "Nitrate (mg/L)": 20,
        "TOC (mg/L)": 1,
        "UVA254 (1/m)": 4,
    },
    "Prealpi": {
        "Turbidity (NTU)": 0.7,
        "UVA254 (1/m)": 1.5,
    },
    "Tabacchi": {
        "Nitrate (mg/L)": 14,
        "Temperature (°C)": 10,
        "Conductivity (uS/cm)": 400,
        "pH": 6,
        
    },
    "Tognazzi": {
        "Conductivity (uS/cm)": 400,
        "Free Chlorine (mg/L)": 0.4,
    },
}

In [ ]:
# TODO fixare questo poichè ora con Tabacchi non c'è più consistenza tra variabili e lower thrshold e upper threshold

figsize = (30, 20)
plt.rcParams.update({"font.size": 22})

for code in sensor_dict.keys():
    
    sensor_df = sensor_dict[code]
    for column in sensor_df.columns:
        df = sensor_df[column].copy()

        # drop rows with duplicated index
        df = df[~df.index.duplicated(keep="first")]

        # plot the data with the thresholds for the variables that have them
        # and compare the distribution of the values with the thresholds

        if column in thresholds[code]:
            threshold = thresholds[code][column]
            fig, ax = plt.subplots(2, 2, figsize=figsize)
            sns.lineplot(x=df.index, y=df, ax=ax[0, 0])
            ax[0, 0].set_title(f"Raw Data")
            ax[0, 0].set_ylabel(column)
            ax[0, 0].set_xlabel("DateTime")
            ax[0, 0].grid()

            fig_hist = sns.histplot(
                df, bins=50, kde=True, stat="probability", ax=ax[1, 0]
            )
            ax[1, 0].set_title(f"Raw Data")
            ax[1, 0].set_ylabel("Probability")
            ax[1, 0].set_xlabel(column)
            ax[1, 0].grid()

            if (column not in ["Conductivity (uS/cm)", "Nitrate (mg/L)"]) and (code != "Tabacchi"):
                ax[0, 0].axhline(
                    y=threshold,
                    color="r",
                    linestyle="dashed",
                    label="Upper Threshold",
                )
                ax[0, 0].text(
                    df.index[0],
                    threshold,
                    f"Upper Threshold: {threshold}",
                    color="r",
                    va="bottom",
                )
                ax[1, 0].axvline(
                    x=threshold,
                    color="r",
                    linestyle="dashed",
                    label="Upper Threshold",
                )
                ax[1, 0].text(
                    threshold,
                    fig_hist.get_ylim()[1],
                    f"Upper Threshold: {threshold}",
                    color="r",
                    rotation=90,
                    ha="right",
                    va="top",
                )
                df = df[df <= threshold]
            else:
                ax[0, 0].axhline(
                    y=threshold,
                    color="r",
                    linestyle="dashed",
                    label="Lower Threshold",
                )
                ax[0, 0].text(
                    df.index[0],
                    threshold,
                    f"Lower Threshold: {threshold}",
                    color="r",
                    va="bottom",
                )
                ax[1, 0].axvline(
                    x=threshold,
                    color="r",
                    linestyle="dashed",
                    label="Lower Threshold",
                )
                ax[1, 0].text(
                    threshold,
                    fig_hist.get_ylim()[1],
                    f"Lower Threshold: {threshold}",
                    color="r",
                    rotation=90,
                    ha="right",
                    va="top",
                )
                df = df[df >= threshold]

            sns.lineplot(x=df.index, y=df, ax=ax[0, 1], color="g")
            ax[0, 1].set_title(f"Filtered Data")
            ax[0, 1].set_ylabel(column)
            ax[0, 1].set_xlabel("DateTime")
            ax[0, 1].grid()

            sns.histplot(
                df,
                bins=50,
                kde=True,
                stat="probability",
                ax=ax[1, 1],
                color="g",
            )
            ax[1, 1].set_title(f"Filtered Data")
            ax[1, 1].set_ylabel("Probability")
            ax[1, 1].set_xlabel(column)
            ax[1, 1].grid()
        else:
            plt.figure(figsize=figsize)
            sns.lineplot(x=df.index, y=df)
            plt.ylabel(column)
            plt.xlabel("DateTime")
            plt.grid()

        plt.suptitle(f"{code} - {column}", fontsize=30)
        plt.tight_layout()

        column_ = column.replace("/", "_")

        path = os.path.join(plot_folder, "Clean Data", "Removed Outliers", code)

        if not os.path.exists(path):
            os.makedirs(path)

        # plt.savefig(
        #     os.path.join(
        #         plot_folder,
        #         "Clean Data",
        #         "Removed Outliers",
        #         code,
        #         f"{column_}.png",
        #     ),
        #     dpi=300,
        # )
        # plt.close()

        plt.show()

plt.rcParams.update({"font.size": 10})

In [ ]:
# remove the rows that have values outside the thresholds
for code in thresholds.keys():
    sensor_df = sensor_dict[code].copy()

    for column in thresholds[code].keys():
        threshold = thresholds[code][column]
        df = sensor_df[column].copy()

        df = (
            df[df > threshold]
            if column not in ["Conductivity (uS/cm)", "Nitrate (mg/L)"]
            else df[df < threshold]
        )

        sensor_df.loc[df.index, column] = np.nan

    sensor_df.interpolate(method="time", inplace=True)

    sensor_dict.update({code: sensor_df})

# Store Data

In [ ]:
grab_df.to_excel(os.path.join(clean_data_folder, "grab.xlsx"), index=False)

In [ ]:
if not os.path.exists(os.path.join(clean_data_folder, "sensors")):
    os.mkdir(os.path.join(clean_data_folder, "sensors"))

for code in sensor_dict.keys():
    sensor_dict[code].to_excel(
        os.path.join(clean_data_folder, "sensors", f"{code}.xlsx"), index=True
    )